In [ ]:
CLUSTER_TYPE ='local'

In [ ]:
from pyspark.sql import SparkSession

# if Spark is run either in Local of Single-Container mode
if CLUSTER_TYPE in ['local', 'docker_container']:
    
    # build a SparkSession 
    #   connect to the master node (address `localhost`) and the port where the master node is listening (7077)
    #   declare the app name 
    #   either connect or create a new context
    spark = SparkSession.builder \
        .master("spark://localhost:7077")\
        .appName("Clustering using K-Means")\
        .getOrCreate()

# if Spark is run as Docker Container cluster (with docker-compose)
elif CLUSTER_TYPE == 'docker_cluster':
    
    # build a SparkSession 
    #   connect to the master node (address `spark-master`) and the port where the master node is listening (7077)
    #   declare the app name 
    #   configure the executor memory to 512 MB
    #   either connect or create a new context
    spark = SparkSession.builder \
        .master("spark://spark-master:7077")\
        .appName("Clustering using K-Means")\
        .config("spark.executor.memory", "512m")\
        .getOrCreate()
else:
    print("Variable CLUSTER_TYPE is not set.")

In [ ]:
spark

In [ ]:
#SparkContext is created by default with the variable name sc
sc = spark.sparkContext
sc

In [ ]:
#running spark default settings
from pyspark import SparkContext
#from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import StandardScaler
from pyspark.mllib.clustering import KMeans, KMeansModel

#print (pyspark.__version__)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs
from numpy import array
from math import sqrt


%matplotlib inline

In [ ]:
#generate input data to work with

#using scikit-learn method to generate N (=10) datasets of dimensionality (d=3)
n_samples=100000
N=10
dim=3
X, y = make_blobs(n_samples=n_samples, centers=N, n_features=dim, cluster_std=default, shuffle=default, random_state=25)

#add id column string for recognition
pddf = pd.DataFrame(X, columns=['x', 'y', 'z'])
pddf['id'] = 'row'+pddf.index.astype(str)

cols = list(pddf)
cols.insert(0, cols.pop(cols.index('id')))
pddf = pddf.ix[:, cols]
pddf.head()

#write array of data in .csv file
pddf.to_csv('input.csv', index=False)

In [ ]:
#want to create an SQLContext, why?
#Spark SQL is a  module for structured data processing differs from the basic Spark RDD API because 
#the interfaces provided gives more information about the structure of both the data and computation
sqlContext = SQLContext(sc)
#read data from csv to spark dataframe
FEATURES_COL = ['x', 'y', 'z']
path = 'input.csv'
df = sqlContext.read.csv(path, header=True)
df.show()
#convert data to column of float
for col in df.columns:
    if col in FEATURES_COL:
        df = df.withColumn(col,df[col].cast('float'))
df = df.na.drop()
df.show()
#feature columns in clustering
#store all features as an array of floats stored as a column (features)
vecAssembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
df_kmeans = vecAssembler.transform(df).select('id', 'features')
df_kmeans.show()

In [ ]:
#standardize the data to bring them to a comparable scale

#scale=StandardScaler(inputCol='features',outputCol='standardized')
#data_scale=scale.fit(df_kmeans)
#data_scale_output=data_scale.transform(df_kmeans)
#data_scale_output.show(2)

In [ ]:
#optimize k choice over a fraction of data
cost = np.zeros(30)
for k in range(2,30):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(df_kmeans.sample(False,0.1, seed=12345))
    cost[k] = model.computeCost(df_kmeans)
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,30),cost[2:30])
ax.set_xlabel('k')
ax.set_ylabel('cost')

In [ ]:
#spark.mllib includes a variant of the k-means++ called kmeans|| (parallel)
#data parallelism creates parallelism by partitioning the dataset into smaller partitions,
#result parallelism is based on targeted clusters


# Load and parse the data


# Build the model (cluster the data)
clusters = KMeans.train(parsedData, N, maxIterations=default, initializationMode="k-means||",seed=default,initializationSteps=default,
                       epsilon=default, initialModel="KMeansModel") #kMeansModel to change

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
sameModel = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")